In [ ]:
# pip install yfinance
# pip install python-bcb==0.3.3

In [ ]:
# BIBLIOTECAS

import yfinance as yf
from datetime import date
from bcb import sgs
import pandas as pd

In [ ]:
comeco = "2016-01-01"
final = "2025-06-25"
# final = date.today().strftime("%Y-%m-%d")

# Define a function to load the dataset
def carregar_base(ticker):
    data = yf.download(ticker, comeco, final)
    data = data.rename(columns={'Close': 'Close'})
    data.reset_index(inplace=True)
    return data

# Remover Multindex dos DataFrames
def achatar_colunas(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(c) for c in col if c]).strip() for col in df.columns]
    return df

In [ ]:
df = carregar_base('ABEV3.SA')
df.head()

/tmp/ipython-input-282-3712625326.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA,ABEV3.SA
0,2016-01-04,12.604818,12.985673,12.604818,12.985673,13206900
1,2016-01-05,12.802571,12.831868,12.531579,12.634117,10774200
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100
3,2016-01-07,12.341151,12.685385,12.341151,12.575523,15316400
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000


In [ ]:
df.columns # Nome das colunas

MultiIndex([(  'Date',         ''),
            ( 'Close', 'ABEV3.SA'),
            (  'High', 'ABEV3.SA'),
            (   'Low', 'ABEV3.SA'),
            (  'Open', 'ABEV3.SA'),
            ('Volume', 'ABEV3.SA')],
           names=['Price', 'Ticker'])

In [ ]:
# Carregar Variáveis Extras

# Dólar
dolar = carregar_base('USDBRL=X')
dolar = achatar_colunas(dolar)
dolar = dolar[['Date', 'Close_USDBRL=X']].rename(columns={'Close_USDBRL=X': 'Close_Dolar'})

# Barril de Petróleo Brent
brent = carregar_base('BZ=F')
brent = achatar_colunas(brent)
brent = brent[['Date', 'Close_BZ=F']].rename(columns={'Close_BZ=F': 'Close_Petroleo'})

# Índice da Ibovespa
ibov = carregar_base('^BVSP')
ibov = achatar_colunas(ibov)
ibov = ibov[['Date', 'Close_^BVSP']].rename(columns={'Close_^BVSP': 'Close_Ibovespa'})

# Taxa Selic
selic = sgs.get(11, start=comeco, end=final)
selic = selic.reset_index()
selic.columns = ['Date', 'Close_Selic']

# IPCA Mensal (série 433)
ipca = sgs.get({'Close_IPCA': 433}, start=comeco, end=final)
ipca = ipca.reset_index()
ipca.columns = ['Date', 'Close_IPCA']
ipca['AnoMes'] = pd.to_datetime(ipca['Date']).dt.to_period('M')
df['AnoMes'] = pd.to_datetime(df['Date']).dt.to_period('M')

# Minério (Empresa Mineradora Global)
minerio = carregar_base('RIO')  # Empresa mineradora global
minerio = achatar_colunas(minerio)
minerio = minerio[['Date', 'Close_RIO']].rename(columns={'Close_RIO': 'Close_Minerio'})

# Energia Elétrica no IPCA (código 188)
energia = sgs.get({'energia_ipca': 188}, start=comeco, end=final)
energia = energia.reset_index()
energia.columns = ['Date', 'Close_Energia']
energia['AnoMesE'] = pd.to_datetime(energia['Date']).dt.to_period('M')
df['AnoMesE'] = pd.to_datetime(df['Date']).dt.to_period('M')

# Produção Industrial (Proxy de Consumo) - série 21859
consumo = sgs.get({'Close_Consumo': 21859}, start=comeco, end=final)
consumo = consumo.reset_index()
consumo.columns = ['Date', 'Close_Consumo']
consumo['AnoMesP'] = pd.to_datetime(consumo['Date']).dt.to_period('M')
df['AnoMesP'] = pd.to_datetime(df['Date']).dt.to_period('M')

energia.head(20)

/tmp/ipython-input-282-3712625326.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-282-3712625326.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-282-3712625326.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-282-3712625326.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, comeco, final)
[*********************100%***********************]  1 of 1 completed


,Date,Close_Energia,AnoMesE
0,2016-01-01,1.51,2016-01
1,2016-02-01,0.95,2016-02
2,2016-03-01,0.44,2016-03
3,2016-04-01,0.64,2016-04
4,2016-05-01,0.98,2016-05
5,2016-06-01,0.47,2016-06
6,2016-07-01,0.64,2016-07
7,2016-08-01,0.31,2016-08
8,2016-09-01,0.08,2016-09
9,2016-10-01,0.17,2016-10


In [ ]:
df = achatar_colunas(df)

df = df.merge(dolar, on='Date', how='inner') # o right basicamente remove a linha caso a data nao bata no merge
df = df.merge(brent, on='Date', how='inner')
df = df.merge(ibov, on='Date', how='inner')
df = df.merge(selic, on='Date', how='inner')
df = pd.merge(df, ipca[['AnoMes', 'Close_IPCA']], on='AnoMes', how='left') # usa o left porque o ipca eh dado uma vez por mes, entao é para manter as colunas que estiverem nulas como nulas
df = df.merge(minerio, on='Date', how='left')
df = pd.merge(df, energia[['AnoMesE', 'Close_Energia']], on='AnoMesE', how='left')
df = pd.merge(df, consumo[['AnoMesP', 'Close_Consumo']], on='AnoMesP', how='left')

df.drop(columns=['AnoMes'], inplace=True)
df.drop(columns=['AnoMesE'], inplace=True)
df.drop(columns=['AnoMesP'], inplace=True)

# Preencher os dias com o valor do mês correspondente
df['Close_IPCA'] = df['Close_IPCA'].fillna(method='ffill')
df['Close_Energia'] = df['Close_Energia'].fillna(method='ffill')
df['Close_Consumo'] = df['Close_Consumo'].fillna(method='ffill')

df.head(200)

/tmp/ipython-input-286-1981840928.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Close_IPCA'] = df['Close_IPCA'].fillna(method='ffill')
/tmp/ipython-input-286-1981840928.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Close_Energia'] = df['Close_Energia'].fillna(method='ffill')
/tmp/ipython-input-286-1981840928.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Close_Consumo'] = df['Close_Consumo'].fillna(method='ffill')


,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa,Close_Selic,Close_IPCA,Close_Minerio,Close_Energia,Close_Consumo
0,2016-01-04,12.604818,12.985673,12.604818,12.985673,13206900,3.9491,37.220001,42141.0,0.052531,1.27,13.606011,1.51,88.9
1,2016-01-05,12.802571,12.831868,12.531579,12.634117,10774200,4.0373,36.419998,42419.0,0.052531,1.27,13.720957,1.51,88.9
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100,4.0269,34.230000,41773.0,0.052531,1.27,12.945110,1.51,88.9
3,2016-01-07,12.341151,12.685385,12.341151,12.575523,15316400,4.0199,33.750000,40695.0,0.052531,1.27,12.360831,1.51,88.9
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000,4.0415,33.549999,40612.0,0.052531,1.27,12.006433,1.51,88.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2016-10-21,14.664901,14.746579,14.479269,14.605499,21287800,3.1391,51.779999,64108.0,0.051660,0.26,16.596720,0.17,105.1
196,2016-10-24,14.627771,14.709448,14.583219,14.642621,10105400,3.1489,51.459999,64060.0,0.051660,0.26,16.722950,0.17,105.1
197,2016-10-25,14.501545,14.605498,14.456992,14.568372,9277400,3.1245,50.790001,63866.0,0.051660,0.26,17.227875,0.17,105.1
198,2016-10-26,14.508970,14.568372,14.382741,14.471843,9041700,3.1104,49.980000,63826.0,0.051660,0.26,17.273315,0.17,105.1


In [ ]:
# df.columns = [f"{col[0]}_{col[1]}" if col[1] != '' else col[0] for col in df.columns]
# df.columns = [col.replace('Close_Dolar_USDBRL=X', 'Close_Dolar').replace('Close_Petroleo_BZ=F', 'Close_Petroleo').replace('Close_Ibovespa_^BVSP', 'Close_Ibovespa') for col in df.columns]
# df.columns

In [ ]:
df.tail()

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa,Close_Selic,Close_IPCA,Close_Minerio,Close_Energia,Close_Consumo
2280,2025-06-17,13.47,13.75,13.40,13.73,32452200,5.4908,76.449997,138840.0,0.054266,0.26,57.009998,0.35,99.4
2281,2025-06-18,13.49,13.55,13.35,13.38,37567000,5.4930,76.699997,138717.0,0.054266,0.26,56.939999,0.35,99.4
2282,2025-06-20,13.53,13.55,13.37,13.41,39451400,5.4900,77.010002,137116.0,0.055131,0.26,55.889999,0.35,99.4
2283,2025-06-23,13.56,13.65,13.47,13.50,27562100,5.5184,71.480003,136551.0,0.055131,0.26,57.090000,0.35,99.4
2284,2025-06-24,13.51,13.65,13.43,13.58,26798300,5.4938,67.139999,137165.0,0.055131,0.26,57.189999,0.35,99.4


In [ ]:
df.head(10)

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa,Close_Selic,Close_IPCA,Close_Minerio,Close_Energia,Close_Consumo
0,2016-01-04,12.604818,12.985673,12.604818,12.985673,13206900,3.9491,37.220001,42141.0,0.052531,1.27,13.606011,1.51,88.9
1,2016-01-05,12.802571,12.831868,12.531579,12.634117,10774200,4.0373,36.419998,42419.0,0.052531,1.27,13.720957,1.51,88.9
2,2016-01-06,12.678060,12.802570,12.597495,12.714681,7739100,4.0269,34.230000,41773.0,0.052531,1.27,12.945110,1.51,88.9
3,2016-01-07,12.341151,12.685385,12.341151,12.575523,15316400,4.0199,33.750000,40695.0,0.052531,1.27,12.360831,1.51,88.9
4,2016-01-08,12.502279,12.597494,12.399742,12.399742,10684000,4.0415,33.549999,40612.0,0.052531,1.27,12.006433,1.51,88.9
5,2016-01-11,12.304530,12.663414,12.267910,12.590172,12772700,3.9932,31.549999,39950.0,0.052531,1.27,11.848389,1.51,88.9
6,2016-01-12,12.436364,12.480310,12.289883,12.407068,9573800,4.0504,30.860001,39514.0,0.052531,1.27,11.503572,1.51,88.9
7,2016-01-13,12.326502,12.634115,12.245936,12.538902,10709300,4.0252,30.309999,38944.0,0.052531,1.27,11.402999,1.51,88.9
8,2016-01-14,12.341151,12.407068,12.099455,12.407068,12325600,4.0141,31.030001,39500.0,0.052531,1.27,12.068692,1.51,88.9
9,2016-01-15,12.150724,12.231290,11.711277,12.194669,19870300,4.0059,31.010000,38569.0,0.052531,1.27,11.312004,1.51,88.9


In [ ]:
df.tail(10)

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa,Close_Selic,Close_IPCA,Close_Minerio,Close_Energia,Close_Consumo
2275,2025-06-10,13.80,14.10,13.73,14.07,23640900,5.5570,66.870003,136436.0,0.054266,0.26,59.470001,0.35,99.4
2276,2025-06-11,13.91,13.92,13.63,13.73,22849900,5.5721,69.769997,137128.0,0.054266,0.26,58.830002,0.35,99.4
2277,2025-06-12,13.80,13.89,13.74,13.75,14788400,5.5358,69.360001,137800.0,0.054266,0.26,58.680000,0.35,99.4
2278,2025-06-13,13.64,13.79,13.58,13.68,28248400,5.5373,74.230003,137213.0,0.054266,0.26,58.000000,0.35,99.4
2279,2025-06-16,13.68,13.85,13.59,13.66,27978300,5.5389,73.230003,139256.0,0.054266,0.26,58.310001,0.35,99.4
2280,2025-06-17,13.47,13.75,13.40,13.73,32452200,5.4908,76.449997,138840.0,0.054266,0.26,57.009998,0.35,99.4
2281,2025-06-18,13.49,13.55,13.35,13.38,37567000,5.4930,76.699997,138717.0,0.054266,0.26,56.939999,0.35,99.4
2282,2025-06-20,13.53,13.55,13.37,13.41,39451400,5.4900,77.010002,137116.0,0.055131,0.26,55.889999,0.35,99.4
2283,2025-06-23,13.56,13.65,13.47,13.50,27562100,5.5184,71.480003,136551.0,0.055131,0.26,57.090000,0.35,99.4
2284,2025-06-24,13.51,13.65,13.43,13.58,26798300,5.4938,67.139999,137165.0,0.055131,0.26,57.189999,0.35,99.4


In [ ]:
df.shape # Quantidade de linhas e colunas

(2285, 14)

In [ ]:
df.info() # Quantidade de dados nulos e tipo dos dados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2285 entries, 0 to 2284
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             2285 non-null   datetime64[ns]
 1   Close_ABEV3.SA   2285 non-null   float64       
 2   High_ABEV3.SA    2285 non-null   float64       
 3   Low_ABEV3.SA     2285 non-null   float64       
 4   Open_ABEV3.SA    2285 non-null   float64       
 5   Volume_ABEV3.SA  2285 non-null   int64         
 6   Close_Dolar      2285 non-null   float64       
 7   Close_Petroleo   2285 non-null   float64       
 8   Close_Ibovespa   2285 non-null   float64       
 9   Close_Selic      2285 non-null   float64       
 10  Close_IPCA       2285 non-null   float64       
 11  Close_Minerio    2283 non-null   float64       
 12  Close_Energia    2285 non-null   float64       
 13  Close_Consumo    2285 non-null   float64       
dtypes: datetime64[ns](1), float64(12), int64

In [ ]:
df.describe() # Estatísticas descritivas das colunas numéricas

,Date,Close_ABEV3.SA,High_ABEV3.SA,Low_ABEV3.SA,Open_ABEV3.SA,Volume_ABEV3.SA,Close_Dolar,Close_Petroleo,Close_Ibovespa,Close_Selic,Close_IPCA,Close_Minerio,Close_Energia,Close_Consumo
count,2285,2285.000000,2285.000000,2285.000000,2285.000000,2.285000e+03,2285.000000,2285.000000,2285.000000,2285.000000,2285.000000,2283.000000,2285.000000,2285.000000
mean,2020-10-02 04:20:54.091903744,13.581211,13.741766,13.425944,13.580819,2.318716e+07,4.553637,68.072425,98617.361926,0.034492,0.425965,42.255384,0.423589,100.740569
min,2016-01-04 00:00:00,9.123590,9.272477,8.569390,8.875440,0.000000e+00,3.056100,19.990000,37497.000000,0.007469,-0.680000,10.871402,-0.600000,70.200000
25%,2018-05-22 00:00:00,12.616715,12.754173,12.469438,12.606898,1.410750e+07,3.724100,54.470001,78596.000000,0.024620,0.210000,28.200939,0.150000,94.400000
50%,2020-10-02 00:00:00,13.534889,13.696745,13.380280,13.528751,2.033570e+07,4.915400,69.029999,104089.000000,0.039270,0.380000,41.842037,0.370000,101.400000
75%,2023-02-14 00:00:00,14.449566,14.604705,14.297597,14.456714,2.876560e+07,5.320000,79.699997,118160.000000,0.049037,0.610000,58.611326,0.640000,106.300000
max,2025-06-24 00:00:00,19.003429,19.230499,18.815510,19.097389,1.440625e+08,6.300000,127.980003,140110.000000,0.055131,1.620000,69.016457,1.710000,114.500000
std,NaN,1.620967,1.612576,1.620169,1.617918,1.344850e+07,0.911694,18.305629,24779.536909,0.014990,0.392874,16.635501,0.423353,7.939634
